# Data Selection in Series
## Series as dictionary

In [180]:
import pandas as pd
data = pd.Series([1,99,2,3,4,6],index=['b','c','a','l','l','a'])
data

b     1
c    99
a     2
l     3
l     4
a     6
dtype: int64

- notice that indices can repeat, unexpected!

In [181]:
data[['a','b']]

a    2
a    6
b    1
dtype: int64

- and that fancy indexing returns both matches to 'a' 

We can also use dictionary-like Python expressions and methods to examine the keys/indices and values:

In [182]:
'l' in data, 'q' in data

(True, False)

In [183]:
data.keys()

Index(['b', 'c', 'a', 'l', 'l', 'a'], dtype='object')

In [184]:
data.items(),'  ',list(data.items())

(<zip at 0x7f2d037ee320>,
 '  ',
 [('b', 1), ('c', 99), ('a', 2), ('l', 3), ('l', 4), ('a', 6)])

Series objects can even be modified with a dictionary-like syntax. Just as you can extend a dictionary by assigning to a new key, you can extend a Series by assigning to a new index value:

In [185]:
data['p'] = 2

In [186]:
data

b     1
c    99
a     2
l     3
l     4
a     6
p     2
dtype: int64

## Series as one-dimensional array

In [187]:
# slicing by explicit index

# data['l':'a'] 
# ==> ERROR "Cannot get right slice bound for non-unique label: 'a'"

data['c':'p']
# note that the element with index 'p' is included

c    99
a     2
l     3
l     4
a     6
p     2
dtype: int64

In [188]:
# slicing by implicit integer index
data[1:3]
# notice: upper bound is not 

c    99
a     2
dtype: int64

In [189]:
# masking
data[data>5]

c    99
a     6
dtype: int64

In [190]:
# fancy indexing
data[['p','l']]

p    2
l    3
l    4
dtype: int64

mong these, slicing may be the source of the most confusion. Notice that when slicing with an explicit index (i.e., data['a':'c']), the final index is included in the slice, while when slicing with an implicit index (i.e., data[0:2]), the final index is excluded from the slice.

## Indexers: loc, iloc, and ix
These slicing and indexing conventions can be a source of confusion. For example, if your Series has an explicit integer index, an indexing operation such as data[1] will use the explicit indices, while a slicing operation like data[1:3] will use the implicit Python-style index.

In [191]:
data = pd.Series(['a','b','c'], index=[1,3,5])
data

1    a
3    b
5    c
dtype: object

In [192]:
# EXPLICIT index when indexing
data[1]

'a'

In [193]:
# IMPLICIT index when slicing
data[1:3]

3    b
5    c
dtype: object

Because of this potential confusion in the case of integer indexes, Pandas provides some special indexer attributes that explicitly expose certain indexing schemes. These are not functional methods, but attributes that expose a particular slicing interface to the data in the Series.

First, the **loc** attribute allows indexing and slicing that always references the **explicit index**:

In [194]:
data.loc[1]

'a'

In [195]:
data.loc[1:3]

1    a
3    b
dtype: object

The **iloc** attribute allows indexing and slicing that always references the **implicit Python-style** index:

In [196]:
data.iloc[0],' ', data.iloc[1]

('a', ' ', 'b')

In [197]:
data.iloc[1:3]

3    b
5    c
dtype: object

In [198]:
print( data.iloc[1:3],    '\n\n',     data[1:3] )

3    b
5    c
dtype: object 

 3    b
5    c
dtype: object


In [199]:
data.iloc[1:3] is data[1:3]
# unexpected...

False

In [200]:
%timeit data.iloc[1:3]
%timeit data.ix[1:3]
%timeit data[1:3]

108 µs ± 5.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


/usr/local/bin/start_ipykernel.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


131 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
130 µs ± 583 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


**One guiding principle of Python code is that "explicit is better than implicit."**

The explicit nature of loc and iloc make them very useful in maintaining clean and readable code; especially in the case of integer indexes, I recommend using these both to make code easier to read and understand, and to prevent subtle bugs due to the mixed indexing/slicing convention.

# Data Selection in DataFrame

Recall that a **DataFrame** acts in many ways like a **two-dimensional or structured array**, and in other ways like a **dictionary of Series structures sharing the same index**.

These analogies can be helpful to keep in mind as we explore data selection within this structure.

In [201]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})

data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [202]:
print(data['area'] is data.area,'\n\n')
data['area']

True 




California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [203]:
%timeit data.area
%timeit data['area']

6.22 µs ± 613 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.18 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Though assessing series via **attribute stype** a useful shorthand,keep in mind that it does not work for all cases! For example, if the column names are not strings, or if the column names conflict with methods of the DataFrame, this attribute-style access is not possible. For example, the DataFrame has a pop() method, so data.pop will point to this rather than the "pop" column:

In [204]:
data.pop is data['pop']

False

Like with the Series objects discussed earlier, this dictionary-style syntax can also be used to modify the object, in this case adding a new column:

In [205]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


# DataFrame as two-dimensional array
As mentioned previously, we can also view the DataFrame as an *enhanced two-dimensional array*. We can examine the raw underlying data array using the values attribute:

In [206]:
print( type(data.values) ) 
data.values

<class 'numpy.ndarray'>


array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

In [207]:
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [208]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


In [209]:
# transposing offers a clevery way of assessing single row via explicit key (i.e. an index element)
data.T['California']

area       4.239670e+05
pop        3.833252e+07
density    9.041393e+01
Name: California, dtype: float64

In [210]:
# here's another more natural way of selecting a single raw via explicit key (i.e. and index element)
data[data.index=='California']

,area,pop,density
California,423967,38332521,90.413926


When it comes to indexing of DataFrame objects, however, it is clear that the dictionary-style indexing of columns precludes our ability to simply treat it as a NumPy array. In particular, passing a single index to an array accesses a row:

In [211]:
# this is one ROW
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

In [212]:
# this is a column/Series, instead
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Thus for array-style indexing, we need another convention. Here Pandas again uses the **loc, iloc, and ix indexers** mentioned earlier. Using the iloc indexer, we can index the underlying array as if it is a simple NumPy array (using the implicit Python-style index), but the DataFrame index and column labels are maintained in the result:

In [213]:
data.iloc[:3,:2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


Similarly, using the loc indexer we can index the underlying data in an array-like style but using the explicit index and column names:

In [214]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135
